In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re

In [2]:
MRT_2021 = pd.read_csv('./DATA/臺北捷運每日分時各站OD流量統計資料_202112.csv')

為什麼 2021

In [3]:
MRT_2021[MRT_2021['時段'] == 2]

,日期,時段,進站,出站,人次
9218811,2022-01-01,2,松山機場,松山機場,0
9218812,2022-01-01,2,松山機場,中山國中,0
9218813,2022-01-01,2,松山機場,南京復興,0
9218814,2022-01-01,2,松山機場,忠孝復興,0
9218815,2022-01-01,2,松山機場,大安,0
...,...,...,...,...,...
9232967,2022-01-01,2,新北產業園區,板新,5
9232968,2022-01-01,2,新北產業園區,Y板橋,16
9232969,2022-01-01,2,新北產業園區,新埔民生,14
9232970,2022-01-01,2,新北產業園區,幸福,18


In [11]:
counts, values = np.unique(MRT_2021['時段'], return_counts=True)

fig = px.bar(x = counts, y = values, log_y=True)
fig

首先，限制日期在XXX

In [ ]:
def filter_new_year(df):
    return df[df['日期'] == '2022-01-01']

# MRT_2021.query("`日期` == '2022-01-01'")

再者，由於xxxxx

In [21]:
def remove_alphabets_from_stations(df):
    import re
    pattern = re.compile('[a-zA-Z]')
    df_ = df.copy()
    df_['進站'] = df['進站'].apply(lambda x: re.sub(pattern, '', x))
    return df_


我們想要xxxx

In [23]:
def sum_enter_by_station_time(df):
    return df.groupby(['時段', '進站'])['人次'].sum().reset_index()


,時段,進站,人次
0,2,七張,114
1,2,三和國中,59
2,2,三民高中,68
3,2,三重,152
4,2,三重國小,89
...,...,...,...
467,5,頂埔,28
468,5,頂溪,187
469,5,頭前庄,51
470,5,麟光,15


In [24]:
MRT_newyear_sum = (MRT_2021
                    .pipe(filter_new_year)
                    .pipe(remove_alphabets_from_stations)
                    .pipe(sum_enter_by_station_time)
                )

MRT_newyear_sum

,時段,進站,人次
0,2,七張,114
1,2,三和國中,59
2,2,三民高中,68
3,2,三重,152
4,2,三重國小,89
...,...,...,...
467,5,頂埔,28
468,5,頂溪,187
469,5,頭前庄,51
470,5,麟光,15


In [26]:
MRT_coordinate = pd.read_csv('./DATA/臺北捷運車站出入口座標.csv', encoding = 'big5')

def get_unique_station_geodata(df):
    df['exit'] = df['出入口名稱'].str.split('站出口').str[0]
    df.loc[df['exit'].str.contains('台北車', regex=False), 'exit' ] = '台北車站'
    return df.groupby('exit').first().reset_index()[['exit', '經度', '緯度']]
    

MRT_first_coord = get_unique_station_geodata(MRT_coordinate)
MRT_first_coord

,exit,經度,緯度
0,七張,121.543068,24.975025
1,三和國中,121.486398,25.076586
2,三民高中,121.472714,25.085692
3,三重,121.483303,25.055352
4,三重國小,121.496702,25.070646
...,...,...,...
113,頂埔,121.418336,24.959327
114,頂溪,121.515414,25.012895
115,頭前庄,121.460303,25.039844
116,麟光,121.558606,25.018554


In [27]:
MRT_2021_sum_with_coord = pd.merge(MRT_newyear_sum, MRT_first_coord, left_on= '進站', right_on='exit', how = 'left')

MRT_2021_sum_with_coord

,時段,進站,人次,exit,經度,緯度
0,2,七張,114,七張,121.543068,24.975025
1,2,三和國中,59,三和國中,121.486398,25.076586
2,2,三民高中,68,三民高中,121.472714,25.085692
3,2,三重,152,三重,121.483303,25.055352
4,2,三重國小,89,三重國小,121.496702,25.070646
...,...,...,...,...,...,...
467,5,頂埔,28,頂埔,121.418336,24.959327
468,5,頂溪,187,頂溪,121.515414,25.012895
469,5,頭前庄,51,頭前庄,121.460303,25.039844
470,5,麟光,15,麟光,121.558606,25.018554


In [45]:
fig = px.density_mapbox(MRT_2021_sum_with_coord, lat='緯度', lon='經度', z='人次', radius=50, zoom=12.5,
                        center=dict(lat=25.033671, lon=121.564427),
                        mapbox_style="carto-positron", animation_frame="時段", hover_name="exit",
                        color_continuous_scale='amp')
fig

In [15]:
pd.read_parquet(r"C:\Users\peipeipig\OneDrive\桌面\Python\YoubikeUsagePrediction\project\DATA\MRT\agg_enter_station_station_mrt.pq")

人次
進站   時段          
台北車站 18  6.575462
     17  6.485979
     19  6.460898
     8   6.442758
市政府  18  6.396186
...           ...
小碧潭  3       -inf
新北投  2       -inf
小碧潭  2       -inf
     4       -inf
新北投  3       -inf

[2856 rows x 1 columns]